## Phys 704 Final Project

 ##### Name: Michael Bouliane 
 ##### ID: 20801403

## Setting Up Ising Model Class

Ising hamiltonian defined as:

$$ H = -J\sum_{\langle i,j \rangle} \sigma_{i} \sigma_{j} $$

Where $J=1$ and $\sigma_{i} = \pm 1$. Lattice is a square lattice of size $L \times L$, where spins are initialized in random configurations.

In [111]:
import numpy as np
import matplotlib.pyplot as plt

rng = np.random.default_rng() #used to generate ising lattice and indices for spin flips

class IsingSystem():
    def __init__(self,T=1.,L=5):
        '''Initializes an Ising lattice of size LxL with reciprocal temperature T'''
        
        self.temp = T
        self.lattice_size = L

        self.lattice = rng.choice((-1,1), size=(self.lattice_size, self.lattice_size))
    
    def Energy(self,idx):
        '''Computes the energy of a single spin site at given location
        
        Args:
        idx: Site index of spin

        Returns:
        E: Energy of site computed with Ising hamiltonian'''

        site = self.lattice[idx]

        left = (idx[0], idx[1]-1)
        right = (idx[0], (idx[1]+1)%self.lattice_size)
        top = (idx[0]-1, idx[1])
        bottom = ((idx[0]+1)%self.lattice_size, idx[1])

        E = -site*sum((self.lattice[left], self.lattice[right], self.lattice[top], self.lattice[bottom]))
        
        return(E)

    def Metropolis(self, steps):
        '''Performs \'steps\' number of Metropolis samplings on system
        
        Args:
        steps: Number of MC steps to take'''

        steps = int(steps)
        idxs = rng.integers(0,self.lattice_size,(steps,2)) #random sites for spin flips
        probs = rng.random(size=steps) #array of acceptance probabilities

        for i in range(steps):
            idx = tuple(idxs[i,:])
            Ei = self.Energy(idx)
            Ef = -Ei

            dE = Ef-Ei

            # if final state is lower in energy, accept the spin flip
            if dE <= 0: 
                self.lattice[idx] = -self.lattice[idx]

            else:
                boltz_fact = np.exp(-dE*self.temp) #boltzmann weight of energy difference
                probs[i] = rng.random() #acceptance probability

                # accept the spin flip that increases energy if boltzmann weight > acceptance prob
                if probs[i]<boltz_fact:
                    self.lattice[idx] = -self.lattice[idx]